In [1]:
import pymongo
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
from Press_11_threshold import *
from error_class import Error

In [2]:
def press_11_alert_generator(start,end):
    press = "Press_11"

    sensor_dict = press_11_quantiles(start,end)

    myclient_global = pymongo.MongoClient(host = "128.121.34.13", connect = True )
    press_db = myclient_global[press]
    end = datetime.now()
    start   = end - timedelta(minutes=60)

    min_max = pd.DataFrame()

    for batch in ['BATCH_1','BATCH_2']:
        collection= press_db[batch]
        projection = {}
        projection['_id'] = 0
        projection['Date'] = 1

        for sensor in sensor_dict.keys():
            projection[sensor] = 1

        QUERY = {"Date": {'$gte': start, '$lt':  end}}
        results = collection.find(QUERY,projection)
        df1 = pd.DataFrame(results).set_index('Date')
        describe = df1.de
        describe().T
        describe = np.percentile(df1.values,[0.5,99.5],axis=0)
        describe = pd.DataFrame(describe, columns=df1.columns, index=[5,95])
        # describe = describe[['min','max']]
        min_max = pd.concat([min_max,describe],axis=0)

    # print(min_max)

    error_list = []
    for i in range(len(min_max)):
        name = min_max.columns[i]
        lower = min_max.iloc[i,0]
        upper = min_max.iloc[i,1]
        [lower_limit, upper_limit] = sensor_dict[name]

        if lower < lower_limit and upper > upper_limit:
            lower_error = True
            upper_error = True
        elif lower < lower_limit:
            lower_error = True
            upper_error = False
        elif upper > upper_limit:
            lower_error = False
            upper_error = True
        else:
            lower_error = False
            upper_error = False

        if lower_error or upper_error:
            descr = ""
            if lower_error and upper_error:
                descr = "Sensor registered data both below limit and above upper limit in the last 5 minutes"
            elif lower_error:
                descr = "Sensor registered data below lower limit"
            elif upper_error:
                descr = "Sensor registered data above upper limit"
            error = Error(name,descr)
            error_list.append(error)

    return error_list

                       min          max
F_L_Nut_Vrms       0.00000     0.005500
F_L_Nut_Arms       0.00000     7.400000
F_L_Nut_Temp      93.92000    95.539990
F_R_Nut_Vrms       0.00000     0.005600
F_R_Nut_Arms       0.00000     9.400001
F_R_Nut_Temp      91.76000    93.020000
R_L_Nut_Vrms       0.00000     0.005400
R_L_Nut_Arms       0.00000     5.600000
R_L_Nut_Temp      93.02000    94.460010
R_R_Nut_Vrms       0.00000     0.005300
R_R_Nut_Arms       0.00000     6.500000
R_R_Nut_Temp      93.02000    94.460010
Ram_Adjust_Vrms    0.00000     0.020200
Ram_Adjust_Arms    0.10000     9.700000
Ram_Adjust_Temp   93.74000    95.539990
M_M_Vrms           0.00000     0.011100
M_M_Arms           0.10000    30.400000
M_M_Temp          95.36000   100.940000
Cl_Vrms            0.00000     0.010400
Cl_Arms            0.10000    17.300000
Cl_Temp           91.58000    95.900000
F_L_Pin_Vrms       0.00000     0.008100
F_L_Pin_Arms       0.10000    12.300000
F_L_Pin_Temp      94.28000    95.360000
